In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [29]:
#api_key = "88382916-d527-45b1-bd27-ccce14a9bac6"

url = "https://api.pokemontcg.io/v2/cards"
#headers = {"X-Api-Key": api_key}
page = 1
all_cards = []

for i in range(1):
    params = {"page": page, "pageSize": 250} 
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()["data"]
        if not data:
            break 
        all_cards.extend(data)
        print(data)
        page += 1  
    else:
        print(f"Erreur: {response.status_code}")
        break

print(f"Nombre total de cartes récupérées : {len(all_cards)}")

for card in all_cards[:10]:
    print(f"Nom de la carte: {card['name']}, Set: {card['set']['name']}")


[{'id': 'dp3-1', 'name': 'Ampharos', 'supertype': 'Pokémon', 'subtypes': ['Stage 2'], 'level': '52', 'hp': '130', 'types': ['Lightning'], 'evolvesFrom': 'Flaaffy', 'abilities': [{'name': 'Jamming', 'text': "After your opponent plays a Supporter card from his or her hand, put 1 damage counter on each of your opponent's Pokémon. You can't use more than 1 Jamming Poké-Body each turn.", 'type': 'Poké-Body'}], 'attacks': [{'name': 'Cluster Bolt', 'cost': ['Lightning', 'Colorless', 'Colorless'], 'convertedEnergyCost': 3, 'damage': '70', 'text': "You may discard all Lightning Energy attached to Ampharos. If you do, this attack does 20 damage to each of your opponent's Benched Pokémon that has any Energy cards attached to it. (Don't apply Weakness and Resistance for Benched Pokémon.)"}], 'weaknesses': [{'type': 'Fighting', 'value': '+30'}], 'resistances': [{'type': 'Metal', 'value': '-20'}], 'retreatCost': ['Colorless', 'Colorless', 'Colorless'], 'convertedRetreatCost': 3, 'set': {'id': 'dp3',

In [33]:
variables_to_drop=['attacks',
                   'supertype',
                   'hp',
                   'abilities',
                   'types',
                   'level',
                   'evolvesFrom',
                   'evolvesTo',
                   'weaknesses',
                   'resistances',
                   'retreatCost',
                   'convertedRetreatCost',
                   'flavorText',
                   'legalities',
                    'images',
                    'rules',
                    'regulationMark',
                    'ancientTrait',
                    'number',
                    'set',
                    'subtypes',
                    'tcgplayer',
                    'cardmarket'
                    
]


def get_prices(x):
    if pd.isna(x["tcgplayer"]) or x["rarity"] == "Common" or pd.isna(x["rarity"]):
        return None, None, None
    
    prices = x["tcgplayer"].get("prices", {})
    holofoil_data = prices.get("holofoil", {})
    reverse_holofoil_data = prices.get("reverseHolofoil", {})

    holofoil_price = holofoil_data.get("market", None)
    reverse_holofoil_price = reverse_holofoil_data.get("market", None)

    if (holofoil_price is not None and holofoil_price > 10) or (reverse_holofoil_price is not None and reverse_holofoil_price > 10):
        return x["tcgplayer"].get("url", None), holofoil_price, reverse_holofoil_price
    else:
        return None, None, None
    
    
def filter_holofoil_data(df):
    df[["url", "holofoil_price", "reverse_holofoil_price"]] = df.apply(get_prices, axis=1, result_type="expand")
    df_cleaned = df.dropna(subset=["url"])
    df_cleaned = df_cleaned[(df_cleaned["rarity"] != "Common") & ((df_cleaned["holofoil_price"] > 10) | (df_cleaned["reverse_holofoil_price"] > 10))]
    print(df_cleaned["set"])
    df_cleaned["collection"]=df_cleaned["set"].apply(lambda x:x["name"])
    df_cleaned["series"]=df_cleaned["set"].apply(lambda x:x["series"])
    
    #drop all the variables we used or not
    df_cleaned = df_cleaned.drop(columns=variables_to_drop, errors='ignore')


    return df_cleaned

In [1]:
from scraping_cleaning_data import import_data, get_prices, filter_holofoil_data

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df=import_data()
df=filter_holofoil_data(df)
df

,id,name,rarity,collection,series,holofoil_price,reverse_holofoil_price,url,nationalPokedexNumbers,artist
0,dp3-1,Ampharos,Rare Holo,Secret Wonders,Diamond & Pearl,14.02,11.17,https://prices.pokemontcg.io/tcgplayer/dp3-1,[181],Kouki Saitou
1,ex12-1,Aerodactyl,Rare Holo,Legend Maker,EX,20.76,32.49,https://prices.pokemontcg.io/tcgplayer/ex12-1,[142],Hajime Kusajima
3,ex7-1,Azumarill,Rare Holo,Team Rocket Returns,EX,24.88,15.66,https://prices.pokemontcg.io/tcgplayer/ex7-1,[184],Sumiyoshi Kizuki
4,sm9-1,Celebi & Venusaur-GX,Rare Holo GX,Team Up,Sun & Moon,11.61,NaN,https://prices.pokemontcg.io/tcgplayer/sm9-1,"[3, 251]",Mitsuhiro Arita
5,pl1-2,Blastoise,Rare Holo,Platinum,Platinum,20.01,12.34,https://prices.pokemontcg.io/tcgplayer/pl1-2,[9],Kagemaru Himeno
...,...,...,...,...,...,...,...,...,...,...
218,dp3-5,Flygon,Rare Holo,Secret Wonders,Diamond & Pearl,10.17,5.07,https://prices.pokemontcg.io/tcgplayer/dp3-5,[330],Daisuke Ito
220,ex8-3,Breloom,Rare Holo,Deoxys,EX,16.48,14.50,https://prices.pokemontcg.io/tcgplayer/ex8-3,[286],Hajime Kusajima
227,ex14-2,Blastoise δ,Rare Holo,Crystal Guardians,EX,51.70,85.76,https://prices.pokemontcg.io/tcgplayer/ex14-2,[9],Masakazu Fukuda
238,sm75-3,Charizard,Rare Holo,Dragon Majesty,Sun & Moon,5.45,10.96,https://prices.pokemontcg.io/tcgplayer/sm75-3,[6],Ryota Murayama


In [3]:
for x in df["url"]:
    print (x)

https://prices.pokemontcg.io/tcgplayer/dp3-1
https://prices.pokemontcg.io/tcgplayer/ex12-1
https://prices.pokemontcg.io/tcgplayer/ex7-1
https://prices.pokemontcg.io/tcgplayer/sm9-1
https://prices.pokemontcg.io/tcgplayer/pl1-2
https://prices.pokemontcg.io/tcgplayer/ex3-2
https://prices.pokemontcg.io/tcgplayer/pop7-1
https://prices.pokemontcg.io/tcgplayer/base4-3
https://prices.pokemontcg.io/tcgplayer/ex7-3
https://prices.pokemontcg.io/tcgplayer/dp3-3
https://prices.pokemontcg.io/tcgplayer/base1-1
https://prices.pokemontcg.io/tcgplayer/ex8-1
https://prices.pokemontcg.io/tcgplayer/pop5-1
https://prices.pokemontcg.io/tcgplayer/base4-1
https://prices.pokemontcg.io/tcgplayer/ex11-1
https://prices.pokemontcg.io/tcgplayer/ex9-1
https://prices.pokemontcg.io/tcgplayer/ex9-2
https://prices.pokemontcg.io/tcgplayer/pl1-1
https://prices.pokemontcg.io/tcgplayer/pl3-2
https://prices.pokemontcg.io/tcgplayer/ex11-2
https://prices.pokemontcg.io/tcgplayer/base4-2
https://prices.pokemontcg.io/tcgplayer/ex7

In [10]:
import requests

url = 'https://prices.pokemontcg.io/tcgplayer/ex11-2'

response = requests.get(url)

if response.status_code == 200:
    # Afficher le contenu HTML brut
    html_content = response.text
    soup = BeautifulSoup(response.text, 'html.parser')
    print(html_content)
    table = soup.find('table', {'data-v-323d0066': True})

    print(table)
else:
    print(f"Erreur : {response.status_code}")


<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8" />
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover" />
    <meta name="version-build" content="2970">
    <meta name="version-hash" content="2d07707b3c1ab7827d55ef6389adccb8bd975698">
    <meta name="version-name" content="origin/hotfix-cm-2911">
    <meta name="facebook-domain-verification" content="ujhbh9jn9ccwotzunt4vehh5l26w5i" />
    <script type="text/javascript">
      ;window.NREUM||(NREUM={});NREUM.init={distributed_tracing:{enabled:true},privacy:{cookies_enabled:true},ajax:{deny_list:["bam-cell.nr-data.net"]}};

      ;NREUM.loader_config={accountID:"2099792",trustKey:"2099792",agentID:"153290625",licenseKey:"a7f40de1df",applicationID:"153290625"};
      ;NREUM.info={beacon:"bam-cell.nr-data.net",errorBeacon:"bam-cell.nr-data.net",licens

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# Chemin vers ChromeDriver
driver_path = '/chemin/vers/chromedriver'  # Remplacez par votre chemin vers ChromeDriver

# Configurer Selenium WebDriver
driver = webdriver.Chrome(driver_path)

# Charger l'URL
url = 'https://prices.pokemontcg.io/tcgplayer/ex11-2'
driver.get(url)

# Attendre que la page charge JavaScript (ajustez si nécessaire)
time.sleep(5)

# Extraire le HTML complet
html_content = driver.page_source

# Compter les lignes dans le HTML complet
lines = html_content.splitlines()
print(f"Nombre de lignes dans la page HTML complète : {len(lines)}")

# Fermer le navigateur
driver.quit()


AttributeError: 'str' object has no attribute 'capabilities'